# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». Нужно построить модель для задачи классификации (с максимально большим значением accuracy, не менее 0.75), которая выберет подходящий тариф.

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы.

**Цель исследования** — построение модели для задачи классификации с наиболее точным значением accuracy

**Ход исследования:**

Для построения модели предоставлен датасет `users_behavior.csv`. Предобработка данных не требуется. 

Исследование пройдёт в шесть этапов:
 1. Обзор данных
 2. Подготовка выборок
 3. Построение моделей
 4. Выбор наилучшей модели
 5. Проверка моделей на вменяемость
 6. Выводы

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Подготовка-выборок" data-toc-modified-id="Подготовка-выборок-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка выборок</a></span></li><li><span><a href="#Построение-моделей" data-toc-modified-id="Построение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Построение моделей</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Выбор-наилучшей-модели" data-toc-modified-id="Выбор-наилучшей-модели-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Выбор наилучшей модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li></ul></div>

## Обзор данных

Имопртируем бибилиотеки:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

SEED = 42

Считаем датасет и сохраним в переменную:

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('users_behavior.csv')

Выведем на экран первые, последние и случайные пять строк датасета и общую информацию:

In [3]:
display(pd.concat([df.head(), df.sample(5, random_state=SEED), df.tail()]),
        df.info(),
        df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
506,46.0,338.60,35.0,11428.54,0
2513,39.0,242.71,0.0,20480.11,0
354,39.0,258.02,0.0,19998.80,0
1080,36.0,230.99,19.0,23525.07,1
2389,35.0,205.35,52.0,35177.94,1


None

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


В датасете `users_behavior.csv` 5 колонок и 3214 строк. Тип данных в колонках `float64` и `int64`.
Согласно документации к данным:
* `сalls` — количество звонков;
* `minutes` — суммарная длительность звонков в минутах;
* `messages` — количество sms-сообщений;
* `mb_used` — израсходованный интернет-трафик в Мб;
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

Пропущенные значения отсутствуют.

**Выводы**

В каждой строке датасета `users_behavior.csv`содержится информация о поведении уникального пользователя за месяц: количество звонков, суммарная длительность звонков в минутах, количество sms-сообщений, израсходованный интернет-трафик в Мб и тариф.

Пропущенные значения отсутствуют. Средние, медианные, максимальные и минимальные значения выглядят правдоподобно, выбивающихся значений нет. Предобработка данных не требуется. Можно приступать к исследованию.

## Подготовка выборок

Проверим баланс классов:

In [4]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Классы не сбалансированы.

В колонке `is_ultra` находится целевой признак, сохраним его в переменную `target`. Разделим исходные данные на обучающую, валидационную и тестовую выборки:

In [5]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']
features_train, features_valid_test, target_train, target_valid_test = train_test_split(features,
                                                                                        target,
                                                                                        test_size=0.4,
                                                                                        random_state=SEED)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test,
                                                                            target_valid_test,
                                                                            test_size=0.5,
                                                                            random_state=SEED)
print('features_train:',features_train.shape, '\n',
      'target_train:', target_train.shape, '\n',
      'features_valid:', features_valid.shape, '\n',
      'target_valid:', target_valid.shape, '\n',
      'features_test:', features_test.shape, '\n',
      'target_test:', target_test.shape)

features_train: (1928, 4) 
 target_train: (1928,) 
 features_valid: (643, 4) 
 target_valid: (643,) 
 features_test: (643, 4) 
 target_test: (643,)


Данные разделены на выборки следующим образом: обучающая - 60%, валидационная - 20%, тестовая - 20%.

## Построение моделей
### Дерево решений
Построим модель классификации с помощью алгоритма дерева решений. Рассмотрим модели с гиперпараметром `max_depth` от 1 до 9:

In [6]:
parameters = {'max_depth': range(1, 10)}
model_tree = DecisionTreeClassifier(random_state=SEED, class_weight='balanced')
grid = GridSearchCV(model_tree, parameters, cv=5, scoring='accuracy')
grid.fit(features_train, target_train)
                                   
model_tree = grid.best_estimator_
print(f'Max depth: {grid.best_params_["max_depth"]}',
      f'Accuracy: {round(grid.best_score_, 3)}',
      sep='\n')

Max depth: 7
Accuracy: 0.778


### Случайный лес
Построим модель классификации с помощью алгоритма случайного леса. Рассмотрим модели с гиперпараметрами `n_estimators` от 30 до 80 с шагом 10 и `max_depth` от 3 до 9:

In [7]:
parameters = {'max_depth': range(3, 10),
              'n_estimators': range(30, 81, 10)}
model_rf = RandomForestClassifier(random_state=SEED, class_weight='balanced')
grid = GridSearchCV(model_rf, parameters, cv=5, scoring='accuracy')
grid.fit(features_train, target_train)
                                   
model_rf = grid.best_estimator_
print(f'Max depth: {grid.best_params_["max_depth"]}',
      f'N estimators: {grid.best_params_["n_estimators"]}',
      f'Accuracy: {round(grid.best_score_, 3)}',
      sep='\n')

Max depth: 8
N estimators: 70
Accuracy: 0.806


### Логистическая регрессия
Построим модель классификации с помощью алгоритма логистической регрессии:

In [8]:
model_lr = LogisticRegression(random_state=SEED, solver='liblinear', class_weight='balanced')
model_lr.fit(features_train, target_train)
scores = cross_val_score(model_lr, features_train, target_train, cv=5, scoring='accuracy')
final_score = scores.mean()
print(f'Accuracy: {round(final_score, 3)}')

Accuracy: 0.628


**Выводы**

- Самое качественное дерево решений с глубиной 7. Accuracy составляет примерно 0.778.
- Самый качественный лес с гиперпараметрами: глубина 8 узлов, количество деревьев 70. Accuracy составляет примерно 0.806.
- Accuracy логистической регрессии составляет примерно 0.628.

### Выбор наилучшей модели
Сравним accuracy моделей на тестовой выборке:

In [9]:
print(f'Decision tree accuracy: {round(model_tree.score(features_test, target_test), 3)}',
      f'Random forest accuracy: {round(model_rf.score(features_test, target_test), 3)}',
      f'Logistic regression accuracy: {round(model_lr.score(features_test, target_test), 3)}',
      sep='\n')

Decision tree accuracy: 0.804
Random forest accuracy: 0.804
Logistic regression accuracy: 0.603


На тестовой выборке наилучший результат показали модели решающего дерева и случайного леса. Accuracy составила примерно 0.804.

### Выводы

При построении моделей разными алгоритмами самые лучшие результаты показали модели со следующими гиперпараметрами:
- Самое качественное дерево решений с глубиной 7. Accuracy составляет примерно 0.778.
- Самый качественный лес с гиперпараметрами: глубина 8, количество деревьев 70. Accuracy составляет примерно 0.806.
- Accuracy логистической регрессии составляет примерно 0.628.

На тестовой выборке наилучший результат показали модели решающего дерева и случайного леса. Accuracy составила примерно 0.804.
Для подбора подходящего тарифа лучше всего подойдет модель случайного леса с гиперпараметрами глубина 8 узлов, количество деревьев 70 или решающее дерево с глубиной 7.